In [23]:
# Ячейка 1: инициализация Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, year, quarter, month, dayofmonth, date_format

spark = SparkSession.builder \
    .appName("Spark ETL Star Schema") \
    .config("spark.jars", "postgresql-42.6.0.jar") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://postgres:5432/spark_db"
pg_props = {
    "user": "spark_user",
    "password": "spark_password",
    "driver": "org.postgresql.Driver"
}

print("✔️ SparkSession запущен")


✔️ SparkSession запущен


In [24]:
# Ячейка 2: читаем сырые данные и считаем строки
df_raw = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "mock_data") \
    .options(**pg_props) \
    .load()

total = df_raw.count()
print(f"📥 Из mock_data прочитано {total} строк")

📥 Из mock_data прочитано 10000 строк


In [25]:
# -------------------------------
# dim_countries
# -------------------------------
print("🔄 Обновление dim_countries...")

# 1) Собираем новые страны из сырых данных
df_cust   = df_raw.select(col("customer_country").alias("country_name"))
df_sell   = df_raw.select(col("seller_country").alias("country_name"))
df_store  = df_raw.select(col("store_country").alias("country_name"))
df_supp   = df_raw.select(col("supplier_country").alias("country_name"))

df_countries_new = (
    df_cust.union(df_sell)
           .union(df_store)
           .union(df_supp)
           .where(col("country_name").isNotNull())
           .distinct()
)
new_cnt = df_countries_new.count()
print(f"   🔍 Найдено {new_cnt} уникальных стран в сырых данных")

# 2) Читаем уже существующие страны из Postgres (правильный синтаксис)
df_exist = spark.read.jdbc(
    url=jdbc_url,
    table="dim_countries",
    properties=pg_props
)

# 3) «Анти-джоин» для определения новых записей
to_insert = df_countries_new.join(df_exist, ["country_name"], "left_anti")
ins_cnt = to_insert.count()
print(f"   ➕ Будут вставлены {ins_cnt} новых строк в dim_countries")

# 4) Запись новых строк (если есть)
if ins_cnt > 0:
    to_insert.write.jdbc(
        url=jdbc_url,
        table="dim_countries",
        mode="append",
        properties=pg_props
    )
    print("   ✅ Вставка в dim_countries завершена")
else:
    print("   ⚠️ Нечего вставлять в dim_countries")


🔄 Обновление dim_countries...
   🔍 Найдено 230 уникальных стран в сырых данных
   ➕ Будут вставлены 0 новых строк в dim_countries
   ⚠️ Нечего вставлять в dim_countries


In [ ]:
# -------------------------------
# dim_cities
# -------------------------------
print("🔄 Обновление dim_cities...")
df_cities_new = (
    df_raw.select(col("store_city").alias("city_name"))
          .union(df_raw.select(col("supplier_city").alias("city_name")))
          .where(col("city_name").isNotNull())
          .distinct()
)
print(f"   🔍 Найдено {df_cities_new.count()} уникальных городов")

df_exist = spark.read.jdbc(url=jdbc_url, table="dim_cities", properties=pg_props)
to_insert = df_cities_new.join(df_exist, ["city_name"], "left_anti")
print(f"   ➕ Будут вставлены {to_insert.count()} новых строк в dim_cities")
if to_insert.count() > 0:
    to_insert.write.jdbc(url=jdbc_url, table="dim_cities", mode="append", properties=pg_props)
    print("   ✅ Вставка в dim_cities завершена")
else:
    print("   ⚠️ Нечего вставлять в dim_cities")

In [ ]:
# -------------------------------
# dim_dates
# -------------------------------
print("🔄 Обновление dim_dates...")
df_dates = (
    df_raw.select(to_date("sale_date",            "MM/dd/yyyy").alias("full_date"))
          .union(df_raw.select(to_date("product_release_date", "MM/dd/yyyy")))
          .union(df_raw.select(to_date("product_expiry_date",  "MM/dd/yyyy")))
          .where(col("full_date").isNotNull())
          .distinct()
          .withColumn("year",    year("full_date"))
          .withColumn("quarter", quarter("full_date"))
          .withColumn("month",   month("full_date"))
          .withColumn("day",     dayofmonth("full_date"))
          .withColumn("weekday", date_format("full_date", "EEEE"))
)
print(f"   🔍 Найдено {df_dates.count()} уникальных дат")

df_exist = spark.read.jdbc(url=jdbc_url, table="dim_dates", properties=pg_props)
to_insert = df_dates.join(df_exist, ["full_date"], "left_anti")
print(f"   ➕ Будут вставлены {to_insert.count()} новых строк в dim_dates")
if to_insert.count() > 0:
    to_insert.write.jdbc(url=jdbc_url, table="dim_dates", mode="append", properties=pg_props)
    print("   ✅ Вставка в dim_dates завершена")
else:
    print("   ⚠️ Нечего вставлять в dim_dates")


In [ ]:
# -------------------------------
# Простые справочники: pet types, breeds, categories
# -------------------------------
dims = [
    ("customer_pet_type",  "dim_pet_types",       "pet_type_name"),
    ("customer_pet_breed", "dim_pet_breeds",      "pet_breed_name"),
    ("pet_category",       "dim_pet_categories",  "pet_category_name"),
]

for col_src, table, name_col in dims:
    print(f"🔄 Обновление {table}...")
    # Извлекаем и переименовываем колонку сразу в правильное имя
    df_new = (
        df_raw
        .select(col(col_src).alias(name_col))
        .where(col(name_col).isNotNull())
        .distinct()
    )
    new_cnt = df_new.count()
    print(f"   🔍 Найдено {new_cnt} уникальных значений в сырых данных")

    df_exist = spark.read.jdbc(url=jdbc_url, table=table, properties=pg_props)
    to_insert = df_new.join(df_exist, [name_col], "left_anti")
    ins_cnt = to_insert.count()
    print(f"   ➕ Будут вставлены {ins_cnt} новых строк в {table}")

    if ins_cnt > 0:
        to_insert.write.jdbc(
            url=jdbc_url,
            table=table,
            mode="append",
            properties=pg_props
        )
        print(f"   ✅ Вставка в {table} завершена")
    else:
        print(f"   ⚠️ Нечего вставлять в {table}")


In [28]:
# -------------------------------
# dim_pets (с выводом найденных записей)
# -------------------------------
print("🔄 Обновление dim_pets...")

# 1) Существующие ключи
df_exist = spark.read.jdbc(
    url=jdbc_url,
    table="dim_pets",
    properties=pg_props
).select("pet_name", "pet_type_id")

# 2) Формируем новые исходные данные и сразу distinct
df_pets_new = (
    df_raw
      .select(
         col("customer_pet_name").alias("pet_name"),
         col("customer_pet_type").alias("pet_type_name"),
         col("customer_pet_breed").alias("pet_breed_name"),
         col("pet_category").alias("pet_category_name")
      )
      .where(col("pet_name").isNotNull())
      .distinct()
)

new_raw_cnt = df_pets_new.count()
print(f"   🔍 В сырых данных найдено {new_raw_cnt} уникальных питомцев (имя+тип)")

# 3) Подтягиваем справочники
dim_pet_types  = spark.read.jdbc(url=jdbc_url, table="dim_pet_types",  properties=pg_props)
dim_pet_breeds = spark.read.jdbc(url=jdbc_url, table="dim_pet_breeds", properties=pg_props)
dim_pet_cats   = spark.read.jdbc(url=jdbc_url, table="dim_pet_categories", properties=pg_props)

df_pets = (
    df_pets_new
      .join(dim_pet_types,  ["pet_type_name"],    "left")
      .join(dim_pet_breeds, ["pet_breed_name"],   "left")
      .join(dim_pet_cats,   ["pet_category_name"],"left")
      .select("pet_name", "pet_type_id", "pet_breed_id", "pet_category_id")
      .distinct()
)

new_cnt = df_pets.count()
print(f"   🔍 После джойна со справочниками и dedupe получено {new_cnt} записей")

# 4) Анти-джоин против уже существующих
to_insert = df_pets.join(df_exist, ["pet_name", "pet_type_id"], "left_anti")
ins_cnt   = to_insert.count()
print(f"   ➕ Будут вставлены {ins_cnt} новых строк в dim_pets")

# 5) Запись (если есть)
if ins_cnt > 0:
    to_insert.write.jdbc(
        url=jdbc_url,
        table="dim_pets",
        mode="append",
        properties=pg_props
    )
    print("   ✅ Вставка в dim_pets завершена")
else:
    print("   ⚠️ Нечего вставлять в dim_pets")


🔄 Обновление dim_pets...
   🔍 В сырых данных найдено 9850 уникальных питомцев (имя+тип)
   🔍 После джойна со справочниками и dedupe получено 9850 записей
   ➕ Будут вставлены 0 новых строк в dim_pets
   ⚠️ Нечего вставлять в dim_pets


In [30]:
# -------------------------------
# dim_suppliers (исправленный блок)
# -------------------------------
print("🔄 Обновление dim_suppliers...")

# 1) Читаем существующие ключи (по уникальному email)
df_exist = spark.read.jdbc(
    url=jdbc_url,
    table="dim_suppliers",
    properties=pg_props
).select("supplier_email")

# 2) Формируем новые записи из сырых данных
df_supp_new = (
    df_raw
      .select(
         col("supplier_name"),
         col("supplier_contact").alias("contact_person"),  # правильно переименовали
         col("supplier_email"),
         col("supplier_phone"),
         col("supplier_address"),
         col("supplier_city").alias("city_name"),
         col("supplier_country").alias("country_name")
      )
      .where(col("supplier_email").isNotNull())
      .distinct()
)

new_cnt = df_supp_new.count()
print(f"   🔍 В сырых данных найдено {new_cnt} уникальных поставщиков по email")

# 3) Подтягиваем внешние ключи: city_id и country_id
dim_cities    = spark.read.jdbc(url=jdbc_url, table="dim_cities",    properties=pg_props)
dim_countries = spark.read.jdbc(url=jdbc_url, table="dim_countries", properties=pg_props)

df_supp = (
    df_supp_new
      .join(dim_cities,    df_supp_new.city_name    == dim_cities.city_name,       "left")
      .join(dim_countries, df_supp_new.country_name == dim_countries.country_name, "left")
      .select(
          "supplier_name",
          "contact_person",
          "supplier_email",
          "supplier_phone",
          "supplier_address",
          col("city_id"),
          col("country_id")
      )
)

post_join_cnt = df_supp.count()
print(f"   🔍 После джойна со справочниками получено {post_join_cnt} записей для вставки")

# 4) Анти-джоин против уже существующих по supplier_email
to_insert = df_supp.join(df_exist, ["supplier_email"], "left_anti")
ins_cnt   = to_insert.count()
print(f"   ➕ Будут вставлены {ins_cnt} новых строк в dim_suppliers")

# 5) Запись (если есть)
if ins_cnt > 0:
    to_insert.write.jdbc(
        url=jdbc_url,
        table="dim_suppliers",
        mode="append",
        properties=pg_props
    )
    print("   ✅ Вставка в dim_suppliers завершена")
else:
    print("   ⚠️ Нечего вставлять в dim_suppliers")


🔄 Обновление dim_suppliers...
   🔍 В сырых данных найдено 10000 уникальных поставщиков по email
   🔍 После джойна со справочниками получено 10000 записей для вставки
   ➕ Будут вставлены 10000 новых строк в dim_suppliers
   ✅ Вставка в dim_suppliers завершена


In [31]:
# -------------------------------
# Продуктовые справочники: category/color/size/brand/material
# -------------------------------
for src,col_name,table in [
    ("product_category","category_name","dim_product_categories"),
    ("product_color",   "color_name",   "dim_product_colors"),
    ("product_size",    "size_name",    "dim_product_sizes"),
    ("product_brand",   "brand_name",   "dim_product_brands"),
    ("product_material","material_name","dim_product_materials"),
]:
    print(f"🔄 Обновление {table}...")
    df_new = (
        df_raw.select(col(src).alias(col_name))
              .where(col(col_name).isNotNull())
              .distinct()
    )
    print(f"   🔍 Найдено {df_new.count()} уникальных значений")

    df_exist = spark.read.jdbc(url=jdbc_url, table=table, properties=pg_props)
    to_insert = df_new.join(df_exist, [col_name], "left_anti")
    print(f"   ➕ Будут вставлены {to_insert.count()} новых строк в {table}")
    if to_insert.count() > 0:
        to_insert.write.jdbc(url=jdbc_url, table=table, mode="append", properties=pg_props)
        print(f"   ✅ Вставка в {table} завершена")
    else:
        print(f"   ⚠️ Нечего вставлять в {table}")


🔄 Обновление dim_product_categories...
   🔍 Найдено 3 уникальных значений
   ➕ Будут вставлены 3 новых строк в dim_product_categories
   ✅ Вставка в dim_product_categories завершена
🔄 Обновление dim_product_colors...
   🔍 Найдено 19 уникальных значений
   ➕ Будут вставлены 19 новых строк в dim_product_colors
   ✅ Вставка в dim_product_colors завершена
🔄 Обновление dim_product_sizes...
   🔍 Найдено 3 уникальных значений
   ➕ Будут вставлены 3 новых строк в dim_product_sizes
   ✅ Вставка в dim_product_sizes завершена
🔄 Обновление dim_product_brands...
   🔍 Найдено 383 уникальных значений
   ➕ Будут вставлены 383 новых строк в dim_product_brands
   ✅ Вставка в dim_product_brands завершена
🔄 Обновление dim_product_materials...
   🔍 Найдено 11 уникальных значений
   ➕ Будут вставлены 11 новых строк в dim_product_materials
   ✅ Вставка в dim_product_materials завершена


In [33]:
# -------------------------------
# dim_products (исправленный блок)
# -------------------------------
print("🔄 Обновление dim_products...")

# 1) Читаем справочники
dim_prod_cat = spark.read.jdbc(url=jdbc_url, table="dim_product_categories", properties=pg_props)
dim_colors   = spark.read.jdbc(url=jdbc_url, table="dim_product_colors",     properties=pg_props)
dim_sizes    = spark.read.jdbc(url=jdbc_url, table="dim_product_sizes",      properties=pg_props)
dim_brands   = spark.read.jdbc(url=jdbc_url, table="dim_product_brands",     properties=pg_props)
dim_mats     = spark.read.jdbc(url=jdbc_url, table="dim_product_materials",  properties=pg_props)
dim_suppliers= spark.read.jdbc(url=jdbc_url, table="dim_suppliers",          properties=pg_props)

# Читаем dim_dates и даём два разных алиаса
dim_dates_full = spark.read.jdbc(url=jdbc_url, table="dim_dates", properties=pg_props)
dim_dates_rel  = dim_dates_full.select(
    col("date_id").alias("release_date_id"),
    col("full_date").alias("release_full_date")
)
dim_dates_exp  = dim_dates_full.select(
    col("date_id").alias("expiry_date_id"),
    col("full_date").alias("expiry_full_date")
)

# 2) Формируем новые продукты
df_products_raw = (
    df_raw
      .select(
         col("product_name"),
         col("supplier_email"),
         col("product_category").alias("category_name"),
         col("product_price").alias("price"),
         col("product_weight").alias("weight"),
         col("product_color").alias("color_name"),
         col("product_size").alias("size_name"),
         col("product_brand").alias("brand_name"),
         col("product_material").alias("material_name"),
         col("product_description").alias("description"),
         col("product_rating").alias("rating"),
         col("product_reviews").alias("reviews"),
         to_date("product_release_date", "MM/dd/yyyy").alias("release_full_date"),
         to_date("product_expiry_date",  "MM/dd/yyyy").alias("expiry_full_date")
      )
      .where(col("product_name").isNotNull())
      .distinct()
)

# 3) Джоиним все справочники по именам
df_products = (
    df_products_raw
      .join(dim_prod_cat, df_products_raw.category_name == dim_prod_cat.category_name, "left")
      .join(dim_colors,   df_products_raw.color_name    == dim_colors.color_name,   "left")
      .join(dim_sizes,    df_products_raw.size_name     == dim_sizes.size_name,     "left")
      .join(dim_brands,   df_products_raw.brand_name    == dim_brands.brand_name,   "left")
      .join(dim_mats,     df_products_raw.material_name == dim_mats.material_name,  "left")
      .join(dim_suppliers,df_products_raw.supplier_email == dim_suppliers.supplier_email, "left")
      .join(dim_dates_rel, df_products_raw.release_full_date == dim_dates_rel.release_full_date, "left")
      .join(dim_dates_exp, df_products_raw.expiry_full_date  == dim_dates_exp.expiry_full_date,  "left")
      .select(
          col("product_name"),
          col("supplier_id"),
          col("category_id"),
          col("price"),
          col("weight"),
          col("color_id"),
          col("size_id"),
          col("brand_id"),
          col("material_id"),
          col("description"),
          col("rating"),
          col("reviews"),
          col("release_date_id"),
          col("expiry_date_id")
      )
      .distinct()
)

# 4) Анти-джоин, чтобы не вставлять дублирующиеся ключи (product_name, supplier_id)
df_exist = spark.read.jdbc(url=jdbc_url, table="dim_products", properties=pg_props) \
                 .select("product_name", "supplier_id")

to_insert = df_products.join(df_exist, ["product_name", "supplier_id"], "left_anti")
ins_cnt   = to_insert.count()
print(f"   ➕ Будут вставлены {ins_cnt} новых строк в dim_products")

# 5) Записываем
if ins_cnt > 0:
    to_insert.write.jdbc(
        url=jdbc_url,
        table="dim_products",
        mode="append",
        properties=pg_props
    )
    print("   ✅ Вставка в dim_products завершена")
else:
    print("   ⚠️ Нечего вставлять в dim_products")


🔄 Обновление dim_products...
   ➕ Будут вставлены 10000 новых строк в dim_products
   ✅ Вставка в dim_products завершена


In [36]:
# -------------------------------
# Финальный блок: dim_customers
# -------------------------------
print("🔄 Обновление dim_customers...")

# Существующие emails
df_exist = spark.read.jdbc(
    url=jdbc_url,
    table="dim_customers",
    properties=pg_props
).select("email")

# Исходные клиенты
df_cust_new = (
    df_raw
      .select(
         col("customer_first_name").alias("first_name"),
         col("customer_last_name").alias("last_name"),
         col("customer_age").alias("age"),
         col("customer_email").alias("email"),
         col("customer_country").alias("country_name"),
         col("customer_postal_code").alias("postal_code"),
         col("customer_pet_name").alias("pet_name")
      )
      .where(col("email").isNotNull())
      .distinct()
)

print(f"   🔍 Найдено в сырых данных {df_cust_new.count()} уникальных клиентов (по email)")

# Подтягиваем FK
dim_countries = spark.read.jdbc(url=jdbc_url, table="dim_countries", properties=pg_props)
dim_pets      = spark.read.jdbc(url=jdbc_url, table="dim_pets",      properties=pg_props)

df_cust = (
    df_cust_new
      .join(dim_countries, ["country_name"], "left")
      .join(dim_pets,      ["pet_name"],      "left")
      .select("first_name", "last_name", "age", "email", 
              col("country_id"), "postal_code", col("pet_id"))
)

print(f"   🔍 После джоина со справочниками {df_cust.count()} записей")

# Анти-джоин + удаляем дубли по email
to_insert = (
    df_cust.join(df_exist, ["email"], "left_anti")
           .dropDuplicates(["email"])
)
ins_cnt = to_insert.count()
print(f"   ➕ Будут вставлены {ins_cnt} новых строк в dim_customers")

# Запись
if ins_cnt > 0:
    to_insert.write.jdbc(
        url=jdbc_url,
        table="dim_customers",
        mode="append",
        properties=pg_props
    )
    print("   ✅ Вставка в dim_customers завершена")
else:
    print("   ⚠️ Нечего вставлять в dim_customers")


🔄 Обновление dim_customers...
   🔍 Найдено в сырых данных 10000 уникальных клиентов (по email)
   🔍 После джоина со справочниками 16581 записей
   ➕ Будут вставлены 10000 новых строк в dim_customers
   ✅ Вставка в dim_customers завершена


In [38]:
# -------------------------------
# dim_sellers (исправленный блок)
# -------------------------------
print("🔄 Обновление dim_sellers...")

# 1) Существующие emails продавцов
df_exist = spark.read.jdbc(
    url=jdbc_url,
    table="dim_sellers",
    properties=pg_props
).select("email")

# 2) Исходный DataFrame продавцов
df_sellers_new = (
    df_raw
      .select(
         col("seller_first_name").alias("first_name"),
         col("seller_last_name").alias("last_name"),
         col("seller_email").alias("email"),
         col("seller_country").alias("country_name"),
         col("seller_postal_code").alias("postal_code")
      )
      .where(col("email").isNotNull())
      .distinct()
)

new_cnt = df_sellers_new.count()
print(f"   🔍 Найдено {new_cnt} уникальных продавцов (по email)")

# 3) Подтягиваем внешний ключ country_id
dim_countries = spark.read.jdbc(url=jdbc_url, table="dim_countries", properties=pg_props)

df_sellers = (
    df_sellers_new
      .join(dim_countries, ["country_name"], "left")
      .select("first_name", "last_name", "email", col("country_id"), "postal_code")
)

post_join_cnt = df_sellers.count()
print(f"   🔍 После джоина со справочниками получено {post_join_cnt} записей")

# 4) Анти-джоин и удаление дубликатов по email
to_insert = (
    df_sellers
      .join(df_exist, ["email"], "left_anti")
      .dropDuplicates(["email"])
)
ins_cnt = to_insert.count()
print(f"   ➕ Будут вставлены {ins_cnt} новых строк в dim_sellers")

# 5) Запись
if ins_cnt > 0:
    to_insert.write.jdbc(
        url=jdbc_url,
        table="dim_sellers",
        mode="append",
        properties=pg_props
    )
    print("   ✅ Вставка в dim_sellers завершена")
else:
    print("   ⚠️ Нечего вставлять в dim_sellers")


🔄 Обновление dim_sellers...
   🔍 Найдено 10000 уникальных продавцов (по email)
   🔍 После джоина со справочниками получено 10000 записей
   ➕ Будут вставлены 10000 новых строк в dim_sellers
   ✅ Вставка в dim_sellers завершена


In [40]:
# -------------------------------
# dim_stores (исправленный блок)
# -------------------------------
print("🔄 Обновление dim_stores...")

# 1) Читаем существующие ключи (store_name + location)
df_exist = spark.read.jdbc(
    url=jdbc_url,
    table="dim_stores",
    properties=pg_props
).select("store_name", "location")

# 2) Собираем новые записи
df_stores_new = (
    df_raw
      .select(
         col("store_name"),
         col("store_location").alias("location"),
         col("store_city").alias("city_name"),
         col("store_state").alias("state"),
         col("store_country").alias("country_name"),
         col("store_phone").alias("phone"),
         col("store_email").alias("email")
      )
      .where(col("store_name").isNotNull())
      .distinct()
)

new_cnt = df_stores_new.count()
print(f"   🔍 В сырых данных найдено {new_cnt} уникальных магазинов (по name+location)")

# 3) Подтягиваем FK city_id и country_id
dim_cities    = spark.read.jdbc(url=jdbc_url, table="dim_cities",    properties=pg_props)
dim_countries = spark.read.jdbc(url=jdbc_url, table="dim_countries", properties=pg_props)

df_stores = (
    df_stores_new
      .join(dim_cities,    ["city_name"],    "left")
      .join(dim_countries, ["country_name"], "left")
      .select(
          "store_name",
          "location",
          col("city_id"),
          "state",
          col("country_id"),
          "phone",
          "email"
      )
)

post_join_cnt = df_stores.count()
print(f"   🔍 После джоина со справочниками получено {post_join_cnt} записей")

# 4) Анти-джоин и удаление дубликатов по ключу
to_insert = (
    df_stores
      .join(df_exist, ["store_name", "location"], "left_anti")
      .dropDuplicates(["store_name", "location"])
)
ins_cnt = to_insert.count()
print(f"   ➕ Будут вставлены {ins_cnt} новых строк в dim_stores")

# 5) Запись
if ins_cnt > 0:
    to_insert.write.jdbc(
        url=jdbc_url,
        table="dim_stores",
        mode="append",
        properties=pg_props
    )
    print("   ✅ Вставка в dim_stores завершена")
else:
    print("   ⚠️ Нечего вставлять в dim_stores")


🔄 Обновление dim_stores...
   🔍 В сырых данных найдено 10000 уникальных магазинов (по name+location)
   🔍 После джоина со справочниками получено 10000 записей
   ➕ Будут вставлены 9688 новых строк в dim_stores
   ✅ Вставка в dim_stores завершена


In [42]:
# -------------------------------
# fact_sales
# -------------------------------
print("🔄 Обновление fact_sales...")

# 1) Читаем и «сужаем» измерения до (ключ, id) и убираем дубликаты
dim_dates = (
    spark.read.jdbc(url=jdbc_url, table="dim_dates", properties=pg_props)
         .select("full_date", "date_id")
         .dropDuplicates(["full_date"])
)

dim_customers = (
    spark.read.jdbc(url=jdbc_url, table="dim_customers", properties=pg_props)
         .select("email", "customer_id")
         .dropDuplicates(["email"])
)

dim_sellers = (
    spark.read.jdbc(url=jdbc_url, table="dim_sellers", properties=pg_props)
         .select("email", "seller_id")
         .dropDuplicates(["email"])
)

dim_products = (
    spark.read.jdbc(url=jdbc_url, table="dim_products", properties=pg_props)
         .select("product_name", "product_id")
         .dropDuplicates(["product_name"])
)

dim_stores = (
    spark.read.jdbc(url=jdbc_url, table="dim_stores", properties=pg_props)
         .select("store_name", "store_id")
         .dropDuplicates(["store_name"])
)

# 2) Подготавливаем сырые данные с правильными колонками для джойна
df_facts = (
    df_raw
      .withColumn("sale_dt", to_date("sale_date", "MM/dd/yyyy"))
      .select(
          "sale_dt",
          "customer_email",
          "seller_email",
          "product_name",
          "store_name",
          col("sale_quantity").alias("quantity"),
          col("sale_total_price").alias("total_price")
      )
)

# 3) Джойним последовательно
df_facts = (
    df_facts
      .join(dim_dates,     df_facts.sale_dt   == dim_dates.full_date, "inner")
      .join(dim_customers, df_facts.customer_email == dim_customers.email, "inner")
      .join(dim_sellers,   df_facts.seller_email   == dim_sellers.email,   "inner")
      .join(dim_products,  df_facts.product_name   == dim_products.product_name, "inner")
      .join(dim_stores,    df_facts.store_name     == dim_stores.store_name,     "inner")
      .select(
          "date_id",
          "customer_id",
          "seller_id",
          "product_id",
          "store_id",
          "quantity",
          "total_price"
      )
)

count = df_facts.count()
print(f"   🔍 После правильных джоинтов найдено {count} фактов")

# 4) Записываем
if count > 0:
    df_facts.write.jdbc(
        url=jdbc_url,
        table="fact_sales",
        mode="append",
        properties=pg_props
    )
    print("   ✅ Вставка в fact_sales завершена")
else:
    print("   ⚠️ Нечего вставлять в fact_sales")



🔄 Обновление fact_sales...
   🔍 После правильных джоинтов найдено 10000 фактов (ожидается ≈10000)
   ✅ Вставка в fact_sales завершена


In [46]:
# Ячейка 1: Инициализация Spark и настройки соединений
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ETL Star Schema Reports to ClickHouse") \
    .config("spark.jars", "postgresql-42.6.0.jar,clickhouse-jdbc-0.4.6.jar") \
    .getOrCreate()

# PostgreSQL
pg_url   = "jdbc:postgresql://postgres:5432/spark_db"
pg_props = {
    "user": "spark_user",
    "password": "spark_password",
    "driver": "org.postgresql.Driver"
}

# ClickHouse
ch_url   = "jdbc:clickhouse://clickhouse:8123/default"
ch_props = {
    "driver":   "com.clickhouse.jdbc.ClickHouseDriver",
    "user":     "default",
    "password": "",
}
print("✔️ SparkSession и JDBC настроены")


✔️ SparkSession и JDBC настроены


In [ ]:
spark.stop()
del spark

In [ ]:
from pyspark import SparkContext
print(SparkContext._active_spark_context)